<a href="https://www.kaggle.com/madeelbadar/diet-affordability-eda?scriptVersionId=89176671" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

****This notebook is a EDA of diet affordability and later turned into a rabbit hole where I explore the relationship between a number of factors that effect public health including purchasing power parity and BMI.****

In [1]:
#Importing all libraries
import pandas as pd
import numpy as np
import plotly.express as px
!pip install country_converter
import country_converter as coco

     |████████████████████████████████| 50 kB 437 kB/s            
  Preparing metadata (setup.py) ... - done
  Created wheel for country-converter: filename=country_converter-0.7.4-py3-none-any.whl size=53543 sha256=d78cfbee9089dfd9c69f633021024212bf2e694f90703540fb0d412eb89f74d7
  Stored in directory: /root/.cache/pip/wheels/69/cf/84/9ebca78737327440a0bb5faa6d0380e9017730534cf0dbcb8c
Successfully built country-converter


In [2]:
#importing file
df=pd.read_csv('../input/affordability-of-dietse/Affordability_of_diets.csv' ,index_col=False)

In [3]:
#dropping any column that contains an N/A value
df=df.dropna(how='any')

In [4]:
#using country_converter library to add an additional column to add ISO3 codes for countries in dataset
#this will help with visualization
entity = coco.convert(names=df['Entity'], to='ISO3')
df['ISO']=entity

In [5]:
#dashboard used to demonstarate the cost of a healthy diet in different countries 
fig = px.choropleth(df, locations="ISO",
                    color='Healthy diet cost  (% cannot afford)', 
                    hover_name="Entity",
                   color_continuous_scale=px.colors.sequential.Plasma)
                    
fig.show()

This is where my curiosity took over and I started to try and uncover relationships between cost of a healthy diet and BMI

In [6]:
#importing another dataset and dropping columns that i do not need 
bmi=pd.read_csv('../input/bmicsv/BMI.csv')
bmi=bmi.drop(columns=['Sex','Country/Region/World'])

In [7]:
#looking at the first 30 rows of the dataframe
bmi.head(30)

,ISO,Year,Mean BMI,Mean BMI lower 95% uncertainty interval,Mean BMI upper 95% uncertainty interval,Prevalence of BMI>=30 kg/m� (obesity),Prevalence of BMI>=30 kg/m� lower 95% uncertainty interval,Prevalence of BMI>=30 kg/m� upper 95% uncertainty interval,Prevalence of BMI>=35 kg/m� (severe obesity),Prevalence of BMI>=35 kg/m� lower 95% uncertainty interval,...,Prevalence of BMI 25 kg/m� to <30 kg/m� upper 95% uncertainty interval,Prevalence of BMI 30 kg/m� to <35 kg/m�,Prevalence of BMI 30 kg/m� to <35 kg/m� lower 95% uncertainty interval,Prevalence of BMI 30 kg/m� to <35 kg/m� upper 95% uncertainty interval,Prevalence of BMI 35 kg/m� to <40 kg/m�,Prevalence of BMI 35 kg/m� to <40 kg/m� lower 95% uncertainty interval,Prevalence of BMI 35 kg/m� to <40 kg/m� upper 95% uncertainty interval,Prevalence of BMI >=40 kg/m�(morbid obesity),Prevalence of BMI >=40 kg/m� lower 95% uncertainty interval,Prevalence of BMI >=40 kg/m� upper 95% uncertainty interval
0,AFG,1975,18.999440,16.331119,21.763077,0.002078,0.000384,0.006515,0.000034,0.000002,...,0.096419,0.002043,0.000351,0.006493,0.000027,6.663263e-07,0.000138,0.000008,5.054664e-08,0.000047
1,AFG,1976,19.105518,16.500077,21.801109,0.002237,0.000437,0.006820,0.000038,0.000002,...,0.097881,0.002199,0.000403,0.006779,0.000030,8.525116e-07,0.000149,0.000009,6.816061e-08,0.000049
2,AFG,1977,19.212080,16.646507,21.842021,0.002410,0.000497,0.007179,0.000042,0.000003,...,0.098869,0.002368,0.000454,0.007144,0.000033,1.080451e-06,0.000163,0.000009,9.255251e-08,0.000053
3,AFG,1978,19.319053,16.786899,21.876516,0.002599,0.000559,0.007567,0.000047,0.000003,...,0.100453,0.002551,0.000519,0.007540,0.000037,1.364412e-06,0.000178,0.000010,1.262377e-07,0.000057
4,AFG,1979,19.425952,16.950002,21.930634,0.002804,0.000636,0.008026,0.000053,0.000004,...,0.102426,0.002751,0.000581,0.007971,0.000041,1.698723e-06,0.000193,0.000011,1.576738e-07,0.000062
5,AFG,1980,19.532923,17.107774,22.010869,0.003026,0.000707,0.008406,0.000059,0.000005,...,0.104154,0.002967,0.000653,0.008336,0.000046,2.092047e-06,0.000214,0.000013,2.004771e-07,0.000069
6,AFG,1981,19.637998,17.232367,22.088491,0.003264,0.000794,0.008856,0.000066,0.000006,...,0.106075,0.003197,0.000730,0.008739,0.000052,2.582559e-06,0.000236,0.000014,2.507258e-07,0.000076
7,AFG,1982,19.741670,17.357383,22.164497,0.003517,0.000875,0.009307,0.000074,0.000007,...,0.108361,0.003443,0.000819,0.009258,0.000058,3.155449e-06,0.000257,0.000016,3.095995e-07,0.000083
8,AFG,1983,19.843253,17.486672,22.266846,0.003787,0.000978,0.009944,0.000084,0.000009,...,0.110522,0.003704,0.000910,0.009859,0.000066,3.918623e-06,0.000282,0.000018,3.800768e-07,0.000093
9,AFG,1984,19.943436,17.650170,22.328421,0.004075,0.001091,0.010396,0.000094,0.000011,...,0.112887,0.003981,0.001014,0.010307,0.000074,4.712529e-06,0.000315,0.000020,4.742994e-07,0.000104


In [8]:
#setting the x value to 2016 
x=(bmi['Year'][41])

In [9]:
#creating a dataframe that only includes the year 2016
bmi=bmi.loc[bmi['Year'] == x]

In [10]:
#creating a data frame that is grouped by the country code value 
bmi=bmi.groupby(['ISO']).mean().reset_index()

In [11]:
#here i am merging the dataset for BMI and the original diet affordability dataset
df=(pd.merge(bmi, df, on='ISO'))

In [12]:
#this figure demonstarted the prevelance of obesity by country
fig = px.choropleth(df, locations="ISO",
                    color='Prevalence of BMI>=30 kg/m� (obesity)', 
                    hover_name="Entity",
                   color_continuous_scale=px.colors.sequential.Plasma)
                    
fig.show()

Let's take a look at the relationship between ppp and health indicators


In [13]:
#importing Purchasing Power Parity file
ppp=pd.read_csv('../input/pppcsv/ppp.csv')

In [14]:
#setting year to 2016
ppp=ppp.loc[ppp['Year'] == 2016]

In [15]:
#renaming column names to make them more understandable 
ppp=ppp.rename(columns={"Country Code": "ISO"})
ppp=ppp.rename(columns={"Value": "Purchasing power parity"})

In [16]:
#merging all the dataframes in order to plot columns within them 
df=(pd.merge(ppp, df, on='ISO'))

In [17]:
#this plot demonstartes the relationship between Mean BMI, cost of a healthy diet and PPP
import plotly.express as px
fig = px.scatter(df, x="Mean BMI", y='Cost of healthy diet (2017 USD per day)', color='Purchasing power parity', trendline='ols', hover_name="Country Name")
fig.show()

In [18]:
#importing the bmi and ppp dataframes again to see the correlation between ppp and bmi over the last 10 years
bmi=pd.read_csv('../input/bmicsv/BMI.csv')
ppp=pd.read_csv('../input/pppcsv/ppp.csv')

In [19]:
#setting the year values for a window between 2006 to 21016
bmi=bmi.loc[(bmi['Year'] >= 2006) & (bmi['Year'] <= 2016)]
ppp=ppp.loc[(ppp['Year'] >= 2006) & (ppp['Year'] <= 2016)]

In [20]:
#showing the first few rows of my dataframe
bmi.head()

,Country/Region/World,ISO,Sex,Year,Mean BMI,Mean BMI lower 95% uncertainty interval,Mean BMI upper 95% uncertainty interval,Prevalence of BMI>=30 kg/m� (obesity),Prevalence of BMI>=30 kg/m� lower 95% uncertainty interval,Prevalence of BMI>=30 kg/m� upper 95% uncertainty interval,...,Prevalence of BMI 25 kg/m� to <30 kg/m� upper 95% uncertainty interval,Prevalence of BMI 30 kg/m� to <35 kg/m�,Prevalence of BMI 30 kg/m� to <35 kg/m� lower 95% uncertainty interval,Prevalence of BMI 30 kg/m� to <35 kg/m� upper 95% uncertainty interval,Prevalence of BMI 35 kg/m� to <40 kg/m�,Prevalence of BMI 35 kg/m� to <40 kg/m� lower 95% uncertainty interval,Prevalence of BMI 35 kg/m� to <40 kg/m� upper 95% uncertainty interval,Prevalence of BMI >=40 kg/m�(morbid obesity),Prevalence of BMI >=40 kg/m� lower 95% uncertainty interval,Prevalence of BMI >=40 kg/m� upper 95% uncertainty interval
31,Afghanistan,AFG,Men,2006,21.869350,19.622649,24.083293,0.018096,0.007525,0.036391,...,0.185815,0.016980,0.006430,0.035422,0.000823,0.000117,0.002691,0.000293,0.000021,0.001188
32,Afghanistan,AFG,Men,2007,21.947270,19.681026,24.218470,0.019275,0.008052,0.038613,...,0.190834,0.018032,0.006911,0.037364,0.000912,0.000132,0.002970,0.000331,0.000025,0.001338
33,Afghanistan,AFG,Men,2008,22.024802,19.749670,24.305962,0.020525,0.008709,0.041075,...,0.195608,0.019140,0.007364,0.039509,0.001010,0.000147,0.003326,0.000375,0.000029,0.001501
34,Afghanistan,AFG,Men,2009,22.102958,19.809451,24.408069,0.021849,0.009361,0.043357,...,0.200424,0.020305,0.007846,0.041675,0.001118,0.000164,0.003663,0.000425,0.000033,0.001683
35,Afghanistan,AFG,Men,2010,22.181569,19.849735,24.505895,0.023247,0.009992,0.046056,...,0.204809,0.021527,0.008338,0.044206,0.001238,0.000183,0.004110,0.000482,0.000039,0.001884


In [21]:
#listing all columns of dataframe
bmi.columns

Index(['Country/Region/World', 'ISO', 'Sex', 'Year', 'Mean BMI',
       'Mean BMI lower 95% uncertainty interval',
       'Mean BMI upper 95% uncertainty interval',
       'Prevalence of BMI>=30 kg/m� (obesity)',
       'Prevalence of BMI>=30 kg/m� lower 95% uncertainty interval',
       'Prevalence of BMI>=30 kg/m� upper 95% uncertainty interval',
       'Prevalence of BMI>=35 kg/m� (severe obesity)',
       'Prevalence of BMI>=35 kg/m� lower 95% uncertainty interval',
       'Prevalence of BMI>=35 kg/m� upper 95% uncertainty interval',
       'Prevalence of BMI<18.5 kg/m� (underweight)',
       'Prevalence of BMI<18.5 kg/m� lower 95% uncertainty interval',
       'Prevalence of BMI<18.5 kg/m� upper 95% uncertainty interval',
       'Prevalence of BMI 18.5 kg/m� to <20 kg/m�',
       'Prevalence of BMI 18.5 kg/m� to <20 kg/m� lower 95% uncertainty interval',
       'Prevalence of BMI 18.5 kg/m� to <20 kg/m� upper 95% uncertainty interval',
       'Prevalence of BMI 20 kg/m� to <25 kg/

In [22]:
#dropping columns I do not need
bmi=bmi.drop(columns=['Sex','Country/Region/World'])

In [23]:
#grouping the dataframe by the values Year and ISO and finding the mean of all other values 
bmi=bmi.groupby(['ISO','Year']).mean().reset_index()

In [24]:
#showing the last 30 values of ppp dataframe
ppp.tail(30)

,Country Name,Country Code,Year,Value
6132,"Yemen, Rep.",YEM,2009,4135.865264
6133,"Yemen, Rep.",YEM,2010,4388.144847
6134,"Yemen, Rep.",YEM,2011,3805.237994
6135,"Yemen, Rep.",YEM,2012,3863.289475
6136,"Yemen, Rep.",YEM,2013,4007.826700
6137,"Yemen, Rep.",YEM,2014,3968.106338
6138,"Yemen, Rep.",YEM,2015,2812.355797
6139,"Yemen, Rep.",YEM,2016,2507.471662
6156,Zambia,ZMB,2006,2392.690039
6157,Zambia,ZMB,2007,2589.897156


In [25]:
#renaming column in order to make it easier for me to work with
ppp=ppp.rename(columns={"Country Code": "ISO"})
ppp=ppp.rename(columns={"Value": "Purchasing power parity"})

In [26]:
#merging columns and creating a new df 
df = ppp.merge(bmi, how='inner', left_on=["ISO", "Year"], right_on=["ISO","Year"])

In [27]:
#specifying the columns I will need and dropping all other columns
df=df[['ISO','Year','Mean BMI','Purchasing power parity']]

In [28]:
#rolling correlation will find correlation between ppp and mean bmi over last 11 periods
df['corr']=df['Purchasing power parity'].rolling(11).corr(df['Mean BMI'])

In [29]:
#dropping all years except 2016
df=df.loc[df['Year']==2016]

In [30]:
#figure demonstarting the corr between mean bmi and ppp for countries 
#not very surprised with the results
fig = px.choropleth(df, locations="ISO",
                    color='corr', 
                    hover_name="ISO",
                   color_continuous_scale=px.colors.sequential.Plasma)
                    
fig.show()